In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_um = df_comuni[df_comuni['cod_regione'] == '10']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20241117/TE/07/PR/'

df_um['url_aff'] = df_um.apply(
    lambda row: f"{BASE_URL}{row['cod_provincia_2']}/{row['tipo']}/{row['cod_comune']}", axis=1
)

df_um['url'] = df_um.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/votanti/italia/{row['cod_regione']}{row['cod_provincia_2']}{row['cod_comune']}", axis=1
)
df_um.sample(3)

/tmp/ipykernel_1759/197730908.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_um['url_aff'] = df_um.apply(
/tmp/ipykernel_1759/197730908.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_um['url'] = df_um.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
382,103040580310,MONTELEONE DI SPOLETO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0310,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...
373,103040580220,GUALDO CATTANEO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0220,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...
366,103040580150,CORCIANO,CM,M,NaN,7,10,UMBRIA,304,TERNI,058,0150,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
print(f"Total URLs to process: {len(df_um['url_aff'])}")

comuni_data = []

for idx, url in enumerate(df_um['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()
    
    provincia = df_um.iloc[idx]['provincia']
    regione = df_um.iloc[idx]['regione']
    url = df_um.iloc[idx]['url']
    print(f"Processing data for Provincia: {provincia}, Regione: {regione}")

    # Get total number of updates available
    n_updates = len(data['enti']['ente_p']['com_vot'])
    print(f"Number of updates available: {n_updates}")

    # Process each update
    for n_update in range(n_updates):
        raw_dt_com = str(data['enti']['ente_p']['com_vot'][n_update]['dt_com'])
        date = raw_dt_com[:8]
        time = raw_dt_com[8:12]

        formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
        formatted_time = f"{time[:2]}:{time[2:]}"
        print(f"Processing update {n_update + 1}/{n_updates} - Timestamp: {formatted_date} {formatted_time}")

        comune_info = {
            'ele_info': data['int']['t_ele'],
            'comune': data['enti']['ente_p']['desc'],
            'provincia': provincia,
            'regione': regione,
            'ele_f': data['enti']['ente_p']['ele_f'],
            'ele_m': data['enti']['ente_p']['ele_m'],
            'ele_t': data['enti']['ente_p']['ele_t'],
            'perc': data['enti']['ente_p']['com_vot'][n_update]['perc'],
            'perc_r': data['enti']['ente_p']['com_vot'][n_update]['perc_r'],
            'vot_t': data['enti']['ente_p']['com_vot'][n_update]['vot_t'],
            'data': formatted_date,
            'ora': formatted_time,
            'update_number': n_update,
            'url': url
        }
        
        print(f"Comune: {comune_info['comune']} - Update {n_update}")
        print(f"Electoral data - Total: {comune_info['ele_t']}, Female: {comune_info['ele_f']}, Male: {comune_info['ele_m']}")
        print(f"Voting percentage: {comune_info['perc']}%, Reporting percentage: {comune_info['perc_r']}%")

        comuni_data.append(comune_info)
    
    print(f"Waiting 0.2 seconds before next request...")
    sleep(0.2)

print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nConverting percentage columns to float...")
df_data['perc_r'] = df_data['perc_r'].str.replace(',', '.').astype(float)
df_data['perc'] = df_data['perc'].str.replace(',', '.').astype(float)

print("\nFinal DataFrame shape:", df_data.shape)
print("\nSample of processed data:")
print(df_data.head())

Total URLs to process: 92

Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20241117/TE/07/PR/058/CM/0010


Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: ASSISI - Update 0
Electoral data - Total: 22980, Female: 11808, Male: 11172
Voting percentage: 9,70%, Reporting percentage: 17,96%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: ASSISI - Update 1
Electoral data - Total: 22980, Female: 11808, Male: 11172
Voting percentage: 33,97%, Reporting percentage: 51,16%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: ASSISI - Update 2
Electoral data - Total: 22980, Female: 11808, Male: 11172
Voting percentage: 41,89%, Reporting percentage: 64,54%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: ASSISI - Update 3
Electoral data - Total: 22980, Female: 11808, Male: 11172
Voting percentage: 56,72%, Reporting percentage: 64,54%
Waiting 0.2 seconds before next request...

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/202

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: BASTIA UMBRA - Update 0
Electoral data - Total: 17033, Female: 8780, Male: 8253
Voting percentage: 9,12%, Reporting percentage: 20,66%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: BASTIA UMBRA - Update 1
Electoral data - Total: 17033, Female: 8780, Male: 8253
Voting percentage: 30,57%, Reporting percentage: 52,17%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: BASTIA UMBRA - Update 2
Electoral data - Total: 17033, Female: 8780, Male: 8253
Voting percentage: 37,68%, Reporting percentage: 64,15%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: BASTIA UMBRA - Update 3
Electoral data - Total: 17033, Female: 8780, Male: 8253
Voting percentage: 51,35%, Reporting percentage: 64,15%
Waiting 0.2 seconds before next request...

Processing record 3
URL: https://eleapi.interno.gov.it/siel/PX

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: BETTONA - Update 0
Electoral data - Total: 3445, Female: 1756, Male: 1689
Voting percentage: 9,11%, Reporting percentage: 17,68%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: BETTONA - Update 1
Electoral data - Total: 3445, Female: 1756, Male: 1689
Voting percentage: 31,35%, Reporting percentage: 53,82%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: BETTONA - Update 2
Electoral data - Total: 3445, Female: 1756, Male: 1689
Voting percentage: 38,55%, Reporting percentage: 66,62%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: BETTONA - Update 3
Electoral data - Total: 3445, Female: 1756, Male: 1689
Voting percentage: 54,31%, Reporting percentage: 66,62%
Waiting 0.2 seconds before next request...

Processing record 4
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20241117/TE

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: BEVAGNA - Update 0
Electoral data - Total: 4239, Female: 2082, Male: 2157
Voting percentage: 6,28%, Reporting percentage: 17,52%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: BEVAGNA - Update 1
Electoral data - Total: 4239, Female: 2082, Male: 2157
Voting percentage: 25,27%, Reporting percentage: 47,43%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: BEVAGNA - Update 2
Electoral data - Total: 4239, Female: 2082, Male: 2157
Voting percentage: 33,38%, Reporting percentage: 61,34%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: BEVAGNA - Update 3
Electoral data - Total: 4239, Female: 2082, Male: 2157
Voting percentage: 47,28%, Reporting percentage: 61,34%
Waiting 0.2 seconds before next request...

Processing record 5
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20241117/TE

...

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: SAN GEMINI - Update 0
Electoral data - Total: 3986, Female: 2062, Male: 1924
Voting percentage: 8,83%, Reporting percentage: 19,50%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: SAN GEMINI - Update 1
Electoral data - Total: 3986, Female: 2062, Male: 1924
Voting percentage: 32,01%, Reporting percentage: 52,84%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: SAN GEMINI - Update 2
Electoral data - Total: 3986, Female: 2062, Male: 1924
Voting percentage: 38,71%, Reporting percentage: 65,15%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: SAN GEMINI - Update 3
Electoral data - Total: 3986, Female: 2062, Male: 1924
Voting percentage: 52,26%, Reporting percentage: 65,15%
Waiting 0.2 seconds before next request...

Processing record 90
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/D

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: SAN VENANZO - Update 0
Electoral data - Total: 1748, Female: 886, Male: 862
Voting percentage: 10,41%, Reporting percentage: 20,13%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: SAN VENANZO - Update 1
Electoral data - Total: 1748, Female: 886, Male: 862
Voting percentage: 38,67%, Reporting percentage: 62,45%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: SAN VENANZO - Update 2
Electoral data - Total: 1748, Female: 886, Male: 862
Voting percentage: 48,05%, Reporting percentage: 75,21%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: SAN VENANZO - Update 3
Electoral data - Total: 1748, Female: 886, Male: 862
Voting percentage: 65,10%, Reporting percentage: 75,21%
Waiting 0.2 seconds before next request...

Processing record 91
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/2

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: STRONCONE - Update 0
Electoral data - Total: 3757, Female: 1875, Male: 1882
Voting percentage: 7,45%, Reporting percentage: 16,49%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: STRONCONE - Update 1
Electoral data - Total: 3757, Female: 1875, Male: 1882
Voting percentage: 26,83%, Reporting percentage: 53,92%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: STRONCONE - Update 2
Electoral data - Total: 3757, Female: 1875, Male: 1882
Voting percentage: 34,04%, Reporting percentage: 67,69%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: STRONCONE - Update 3
Electoral data - Total: 3757, Female: 1875, Male: 1882
Voting percentage: 48,84%, Reporting percentage: 67,69%
Waiting 0.2 seconds before next request...

Processing record 92
URL: https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20

Response status code: 200
Processing data for Provincia: TERNI, Regione: UMBRIA
Number of updates available: 4
Processing update 1/4 - Timestamp: 2024-11-17 12:00
Comune: TERNI - Update 0
Electoral data - Total: 86299, Female: 44992, Male: 41307
Voting percentage: 8,99%, Reporting percentage: 19,71%
Processing update 2/4 - Timestamp: 2024-11-17 19:00
Comune: TERNI - Update 1
Electoral data - Total: 86299, Female: 44992, Male: 41307
Voting percentage: 28,22%, Reporting percentage: 49,96%
Processing update 3/4 - Timestamp: 2024-11-17 23:00
Comune: TERNI - Update 2
Electoral data - Total: 86299, Female: 44992, Male: 41307
Voting percentage: 33,66%, Reporting percentage: 61,68%
Processing update 4/4 - Timestamp: 2024-11-18 15:00
Comune: TERNI - Update 3
Electoral data - Total: 86299, Female: 44992, Male: 41307
Voting percentage: 46,72%, Reporting percentage: 61,68%
Waiting 0.2 seconds before next request...

Creating DataFrame...



Converting percentage columns to float...

Final DataFrame shape: (368, 14)

Sample of processed data:
             ele_info        comune provincia regione  ele_f  ele_m  ele_t  \
0  Regionali Speciali        ASSISI     TERNI  UMBRIA  11808  11172  22980   
1  Regionali Speciali        ASSISI     TERNI  UMBRIA  11808  11172  22980   
2  Regionali Speciali        ASSISI     TERNI  UMBRIA  11808  11172  22980   
3  Regionali Speciali        ASSISI     TERNI  UMBRIA  11808  11172  22980   
4  Regionali Speciali  BASTIA UMBRA     TERNI  UMBRIA   8780   8253  17033   

    perc  perc_r  vot_t        data    ora  update_number  \
0   9.70   17.96   2230  2024-11-17  12:00              0   
1  33.97   51.16   7807  2024-11-17  19:00              1   
2  41.89   64.54   9626  2024-11-17  23:00              2   
3  56.72   64.54  13034  2024-11-18  15:00              3   
4   9.12   20.66   1553  2024-11-17  12:00              0   

                                                 url  
0  ht

In [6]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,ele_info,comune,provincia,regione,ele_f,ele_m,ele_t,perc,perc_r,vot_t,data,ora,update_number,url
0,Regionali Speciali,Assisi,Terni,Umbria,11808,11172,22980,9.70,17.96,2230,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...
1,Regionali Speciali,Assisi,Terni,Umbria,11808,11172,22980,33.97,51.16,7807,2024-11-17,19:00,1,https://elezioni.interno.gov.it/risultati/2024...
2,Regionali Speciali,Assisi,Terni,Umbria,11808,11172,22980,41.89,64.54,9626,2024-11-17,23:00,2,https://elezioni.interno.gov.it/risultati/2024...
3,Regionali Speciali,Assisi,Terni,Umbria,11808,11172,22980,56.72,64.54,13034,2024-11-18,15:00,3,https://elezioni.interno.gov.it/risultati/2024...
4,Regionali Speciali,Bastia Umbra,Terni,Umbria,8780,8253,17033,9.12,20.66,1553,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,Regionali Speciali,Stroncone,Terni,Umbria,1875,1882,3757,48.84,67.69,1835,2024-11-18,15:00,3,https://elezioni.interno.gov.it/risultati/2024...
364,Regionali Speciali,Terni,Terni,Umbria,44992,41307,86299,8.99,19.71,7759,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...
365,Regionali Speciali,Terni,Terni,Umbria,44992,41307,86299,28.22,49.96,24356,2024-11-17,19:00,1,https://elezioni.interno.gov.it/risultati/2024...
366,Regionali Speciali,Terni,Terni,Umbria,44992,41307,86299,33.66,61.68,29044,2024-11-17,23:00,2,https://elezioni.interno.gov.it/risultati/2024...


In [7]:
df_data['perc_diff'] = round(df_data['perc']-df_data['perc_r'],2)
df_data.sample(5)

,ele_info,comune,provincia,regione,ele_f,ele_m,ele_t,perc,perc_r,vot_t,data,ora,update_number,url,perc_diff
76,Regionali Speciali,Fratta Todina,Terni,Umbria,727,667,1394,11.69,18.34,163,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...,-6.65
129,Regionali Speciali,Montone,Terni,Umbria,680,655,1335,36.78,61.33,491,2024-11-17,19:00,1,https://elezioni.interno.gov.it/risultati/2024...,-24.55
345,Regionali Speciali,Polino,Terni,Umbria,81,98,179,39.11,58.21,70,2024-11-17,19:00,1,https://elezioni.interno.gov.it/risultati/2024...,-19.10
328,Regionali Speciali,Orvieto,Terni,Umbria,8280,7512,15792,9.88,19.92,1560,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...,-10.04
288,Regionali Speciali,Ficulle,Terni,Umbria,681,687,1368,6.51,15.63,89,2024-11-17,12:00,0,https://elezioni.interno.gov.it/risultati/2024...,-9.12


In [8]:
df_data.to_csv('../output/affluenze_um.csv', index=False, encoding='UTF-8-sig')

In [9]:
# Find the latest update_number where perc is not 0
latest_update = df_data[df_data['perc'] != 0]['update_number'].max()
print(f"Latest update number with non-zero percentages: {latest_update}")

# Filter DataFrame to only include rows from that update
latest_data = df_data[df_data['update_number'] == latest_update].copy()  # Create a copy to avoid the SettingWithCopyWarning
print(f"\nShape of filtered data: {latest_data.shape}")

# Use .str.replace() to handle string replacement more safely
latest_data['comune'] = latest_data['comune'].str.replace("Citta'", "Città", regex=False)

# Save to CSV
latest_data.to_csv('../output/viz/affluenze_um_latest.csv', index=False, encoding='UTF-8')


Latest update number with non-zero percentages: 3

Shape of filtered data: (92, 15)
